In [1]:
import util as ut
import recomender.trivial as trivial
import evaluate as eval

import json
import numpy as np
from sklearn.model_selection import train_test_split as sk_train_test_split
from operator import itemgetter


In [3]:
# Separate test and train
test, train = ut.read_ratings()

# put test in the same recomendation format
test_recomendation = eval.test_to_recomendation(test)



In [19]:
train_user_dict, train_item_dict, train_ratings_dict = train


In [6]:
with open(ut.CONTENT, 'r') as json_file:
    json_list = list(json_file)


content_dict = {}
for json_str in json_list:
    result = json.loads(json_str)
    if (isinstance(result, dict)):
        content_dict[result['ItemId']] = result   

In [57]:

def make_genre_dict(train_user_dict, content_dict):  
  genre_dict = {}
  for user_id in train_user_dict.keys():
    genre_dict[user_id] = {}
    items = train_user_dict[user_id]['Items']
    for item in items:
      genres = content_dict[item]['Genre'].split(', ')
      for genre in genres:
        if genre not in genre_dict[user_id]: genre_dict[user_id][genre] = []
        genre_dict[user_id][genre].append(item)
  return genre_dict
genre_dict = make_genre_dict(train_user_dict, content_dict)

{'c4ca4238a0': {'Action': ['91766eac45'],
  'Horror': ['91766eac45'],
  'Sci-Fi': ['91766eac45']},
 'c81e728d9d': {'Action': ['5c739554f7', '48f6d7ce7c', 'e9318d627a'],
  'Adventure': ['5c739554f7', '48f6d7ce7c'],
  'Fantasy': ['5c739554f7', 'e9318d627a'],
  'Animation': ['48f6d7ce7c'],
  'Horror': ['e9318d627a']},
 'a87ff679a2': {'Biography': ['17e6357973'],
  'Drama': ['17e6357973',
   '1018e3f80b',
   'c2557bea38',
   'eeff885aad',
   '04d343d55c'],
  'Crime': ['1018e3f80b'],
  'Romance': ['c2557bea38', '04d343d55c'],
  'Thriller': ['04d343d55c']},
 'e4da3b7fbb': {'Drama': ['afe4b38e5a',
   '09d61170b2',
   'b2b9bf57f7',
   '8edac6fbba'],
  'Romance': ['afe4b38e5a', 'c1d7c2545c'],
  'Biography': ['09d61170b2', '8edac6fbba'],
  'Comedy': ['09d61170b2', 'c1d7c2545c', 'b2b9bf57f7'],
  'Crime': ['b2b9bf57f7'],
  'Thriller': ['8edac6fbba']},
 '1679091c5a': {'Drama': ['adad9af659', 'a293af8be0'],
  'Mystery': ['adad9af659'],
  'Romance': ['adad9af659'],
  'Adventure': ['a293af8be0'],
  'F

In [81]:
def rating_by_genre(user, item, ratings, content_dict, genre_dict):

  genre_count = 0
  rate_by_genre = 0
  genres = content_dict[item]['Genre'].split(',')
  for genre in genres:
    if not genre in genre_dict[user]: continue
    n_items = len(genre_dict[user][genre])
    genre_rating = 0 
    for genre_item in genre_dict[user][genre]:
      genre_rating += ratings[user + ':' + genre_item]['Rating']
    genre_rating = genre_rating/n_items
    print(genre_rating)
    rate_by_genre += genre_rating
    genre_count += 1
  
  return rate_by_genre, genre_count

def trivial_personalized_prediction(train, targets, content_dict):
  train_user_dict, train_item_dict, train_ratings_dict = train
  genre_dict = make_genre_dict(train_user_dict, content_dict)

  mean_rating = 0
  for key in train[2]:
    mean_rating += train[2][key]['Rating']
  mean_rating = mean_rating/len(train[2])

  predictions = []
  for pair in targets:
    _tuser, _titem = pair.split(':')
    dif_item_mean = 0
    dif_rating = 0
    weight = 1
    if _titem in train_item_dict:
      _item_dict = train_item_dict[_titem]
      dif_item_mean = _item_dict['Rating_sum']/len(_item_dict['Users']) - mean_rating
    
      
      if _tuser in train_user_dict: 
        dif_rating, weight = rating_by_genre(_tuser, _titem, train_ratings_dict, 
                                              content_dict, genre_dict) 
        dif_rating = dif_rating - mean_rating
    
    print(dif_rating , dif_item_mean)
    prediction = mean_rating + (dif_rating + dif_item_mean)/(weight) 
    predictions.append([_tuser, _titem, prediction])
  
  predictions = sorted(predictions, key=itemgetter(2), reverse=True)  
  return predictions

trivial_personalized_prediction(train, test[2].keys(), content_dict)

0 0
0 0
0 0
6.0
-2.083333333333334 -0.08333333333333393
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


[['e4da3b7fbb', '9b4aeab5c9', 8.083333333333334],
 ['8f14e45fce', '81b1003c67', 8.083333333333334],
 ['45c48cce2e', '7a4217c657', 8.083333333333334],
 ['b6d767d2f8', '594bef2567', 8.083333333333334],
 ['182be0c5cd', 'f9f385e533', 8.083333333333334],
 ['19ca14e7ea', '52e1600389', 8.083333333333334],
 ['d67d8ab4f4', '8b6785454c', 8.083333333333334],
 ['d67d8ab4f4', '9f0dd8a383', 8.083333333333334],
 ['d67d8ab4f4', '6ab6b52e26', 8.083333333333334],
 ['d67d8ab4f4', '0575427f00', 8.083333333333334],
 ['d67d8ab4f4', '79ee451e92', 8.083333333333334],
 ['6f4922f455', 'a32b86ad8b', 5.916666666666666]]